# Leistungsnachweis 
# Modul D Big Data

Dozent: Prof. Dr. Kurt Stockinger


Autoren: Andreas Palm und Reto Stucki


"11. Januar 2019"

Basierend auf einem öffentlich erhältlichen Datensatz über Flugzeugabstürze(1908-2018) in dem zu jedem Flugzeugabsturz eine Kurzbeschreibung des Ereignises festgehalten ist soll versucht werden die Absturzursache zu kategorisieren:

- Andere
- Mechanik
- Pilotenfehler
- Wetter
- Sabotage

Anschliessend sollen die Abstürze nach folgende Kritieren untersucht werden:

- Fluggesellschaft
- Absturzursache
- Flugzeugtype

Dabei soll auch die Entwicklung über den Zeitraum des Datensatzes dargestellt werden.

Bei der Kategorisierung der Absturzursachen soll in einem ersten Anlauf überprüft werden ob mit einem einfachen Ansatz eine hohe Präzision der Kategorisierung erreicht werden kann (> 90%). Falls dies nicht funktioniert, soll evaluiert werden welche weiteren Ansätze zu einer höheren Präzision führen.

* Python 2
* test_helper, siehe [here](https://databricks-staging-cloudfront.staging.cloud.databricks.com/public/c65da9a2fa40e45a2028cddebe45b54c/8637560089690848/4187311313936645/6977722904629137/05f3c2ecc3.html).

**Kaggleprojekt:** https://www.kaggle.com/nguyenhoc/plane-crash

**File:** planecrashinfo_20181121001952.csv

**Stand:** 21.11.2018

**Beschreibung Dataset:**

<table style="width: 618.9px;" border="0" cellspacing="0" cellpadding="2" align="center">
<tbody>
<tr>
<td style="width: 78px;" align="right"><strong><span class="style10">Bezeichnung</span></strong></td>
<td style="width: 525.9px;"><strong><span class="style7">Beschreibung</span></strong></td>
</tr>
<tr>
<td style="width: 78px;" align="right"><span class="style10">Date:</span></td>
<td style="width: 525.9px;"><span class="style7">&nbsp;Date of accident, &nbsp;in the format - January 01, 2001</span></td>
</tr>
<tr>
<td style="width: 78px;" align="right"><span class="style10"> Time:</span></td>
<td style="width: 525.9px;"><span class="style7">&nbsp;Local time, in 24 hr. format unless otherwise specified</span></td>
</tr>
<tr>
<td style="width: 78px;" align="right"><span class="style10"> Airline/Op:</span></td>
<td style="width: 525.9px;"><span class="style7">&nbsp;Airline or operator of the aircraft</span></td>
</tr>
<tr>
<td style="width: 78px;" align="right"><span class="style10"> Flight #:</span></td>
<td style="width: 525.9px;"><span class="style7">&nbsp;Flight number assigned by the aircraft operator</span></td>
</tr>
<tr>
<td style="width: 78px;" align="right"><span class="style10"> Route:</span></td>
<td style="width: 525.9px;"><span class="style7">&nbsp;Complete or partial route flown prior to the accident</span></td>
</tr>
<tr>
<td style="width: 78px;" align="right"><span class="style10"> AC Type:</span></td>
<td style="width: 525.9px;"><span class="style7">&nbsp;Aircraft type</span></td>
</tr>
<tr>
<td style="width: 78px;" align="right"><span class="style10"> Reg:</span></td>
<td style="width: 525.9px;"><span class="style7">&nbsp;ICAO registration of the aircraft</span></td>
</tr>
<tr>
<td style="width: 78px;" align="right"><span class="style10"> cn / ln:</span></td>
<td style="width: 525.9px;"><span class="style7">&nbsp;Construction or serial number / Line or fuselage number</span></td>
</tr>
<tr>
<td style="width: 78px;" align="right"><span class="style10"> Aboard:</span></td>
<td style="width: 525.9px;"><span class="style7">&nbsp;Total aboard (passengers / crew)</span></td>
</tr>
<tr>
<td style="width: 78px;" align="right"><span class="style10"> Fatalities:</span></td>
<td style="width: 525.9px;"><span class="style7">&nbsp;Total fatalities aboard (passengers / crew)</span></td>
</tr>
<tr>
<td style="width: 78px;" align="right"><span class="style10"> Ground:</span></td>
<td style="width: 525.9px;"><span class="style7">&nbsp;Total killed on the ground</span></td>
</tr>
<tr>
<td style="width: 78px;" align="right"><span class="style10"> Summary:</span></td>
<td style="width: 525.9px;"><span class="style7">&nbsp;Brief description of the accident and cause if known</span></td>
</tr>
</tbody>
</table>

Für die Tests wurde die test_helper Klasse integriert.

Für die einzelnen Funktionen wurden Testfälle erstellt.

Um die Resultate sauber zu verifizieren, wurden die Daten auch noch in eine MySQL Datenbank geladen.
Die Abfragen wurden zur Überprüfung jeweils auf der MySQL DB laufen gelassen.

In [6]:
from pyspark.sql.functions import *
from datetime import datetime

# File location and type
file_location = "/FileStore/tables/planecrashinfo_20181121001952.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
# .option("dateFormat", "MMM d, yy") seems not to work
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)


# Hence date conversion has to be done manually
dfcrashes = df.withColumn("date", to_date(df.date, 'MMM d, yy'))
display(dfcrashes)

date,time,location,operator,flight_no,route,ac_type,registration,cn_ln,aboard,fatalities,ground,summary
1908-09-17,17:18,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2 (passengers:1 crew:1),1 (passengers:1 crew:0),0,"During a demonstration flight, a U.S. Army flyer flown by Orville Wright nose-dived into the ground from a height of approximately 75 feet, killing Lt. Thomas E. Selfridge, 26, who was a passenger. This was the first recorded airplane fatality in history. One of two propellers separated in flight, tearing loose the wires bracing the rudder and causing the loss of control of the aircraft. Orville Wright suffered broken ribs, pelvis and a leg. Selfridge suffered a crushed skull and died a short time later."
1909-09-07,?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1 (passengers:0 crew:1),1 (passengers:0 crew:0),0,"Eugene Lefebvre was the first pilot to ever be killed in an air accident, after his controls jambed while flying in an air show."
1912-07-12,06:30,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5 (passengers:0 crew:5),5 (passengers:0 crew:5),0,"First U.S. dirigible Akron exploded just offshore at an altitude of 1,000 ft. during a test flight."
1913-08-06,?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1 (passengers:0 crew:1),1 (passengers:0 crew:1),0,"The first fatal airplane accident in Canada occurred when American barnstormer, John M. Bryant, California aviator was killed."
1913-09-09,c 18:30,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20 (passengers:? crew:?),14 (passengers:? crew:?),0,The airship flew into a thunderstorm and encountered a severe downdraft crashing 20 miles north of Helgoland Island into the sea. The ship broke in two and the control car immediately sank drowning its occupants.
1913-10-17,c 10:30,"Near Johannisthal, Germany",Military - German Navy,?,?,Zeppelin L-2 (airship),?,?,30 (passengers:? crew:?),30 (passengers:? crew:?),0,"Hydrogen gas which was being vented was sucked into the forward engine and ignited causing the airship to explode and burn at 3,000 ft.."
1915-03-05,c 01:00,"Tienen, Belgium",Military - German Navy,?,?,Zeppelin L-8 (airship),?,?,41 (passengers:? crew:?),21 (passengers:? crew:?),0,Crashed into trees while attempting to land after being shot down by British and French aircraft.
1915-09-03,c 15:20,"Off Cuxhaven, Germany",Military - German Navy,?,?,Zeppelin L-10 (airship),?,?,19 (passengers:? crew:?),19 (passengers:? crew:?),0,"Exploded and burned near Neuwerk Island, when hydrogen gas, being vented, was ignited by lightning."
1916-07-28,?,"Near Jambol, Bulgeria",Military - German Army,?,?,Schutte-Lanz S-L-10 (airship),?,?,20 (passengers:? crew:?),20 (passengers:? crew:?),0,"Crashed near the Black Sea, cause unknown."
1916-09-24,c 01:00,"Billericay, England",Military - German Navy,?,?,Zeppelin L-32 (airship),?,?,22 (passengers:? crew:?),22 (passengers:? crew:?),0,Shot down by British aircraft crashing in flames.


In [7]:
temp_table = "crashestable"

dfcrashes.createOrReplaceTempView(temp_table)

In [8]:
def checkanzrecord():
  return dfcrashes.count()

print (checkanzrecord())

5784

In [9]:
# TEST Ob die eingelesenen Records der erwarteten Anzahl Records entspricht
from test_helper import Test
Test.assertEquals(checkanzrecord(), 5784, 'incorrect Total Records')

1 test passed.

In [10]:
%sql

select * from crashestable 
where date is not null
order by date asc, time asc
limit 1

date,time,location,operator,flight_no,route,ac_type,registration,cn_ln,aboard,fatalities,ground,summary
1908-09-17,17:18,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2 (passengers:1 crew:1),1 (passengers:1 crew:0),0,"During a demonstration flight, a U.S. Army flyer flown by Orville Wright nose-dived into the ground from a height of approximately 75 feet, killing Lt. Thomas E. Selfridge, 26, who was a passenger. This was the first recorded airplane fatality in history. One of two propellers separated in flight, tearing loose the wires bracing the rudder and causing the loss of control of the aircraft. Orville Wright suffered broken ribs, pelvis and a leg. Selfridge suffered a crushed skull and died a short time later."


In [11]:
%sql

select * from crashestable 
where date is not null
order by date desc, time desc
limit 1

date,time,location,operator,flight_no,route,ac_type,registration,cn_ln,aboard,fatalities,ground,summary
2018-11-06,0253,"Georgetown, Guyana",Fly Jamaica Airways,?,Georgetown - Toronto,Boeing 757-N23,N524AT,30233/895,128 (passengers:120 crew:8),1 (passengers:1 crew:0),0,"After taking off and reaching FL200, the crew reported a hydraulic problem and circled back to Georgetown Airport. The aircraft touched down on runway 06 but went off the runway and came to rest against the airport perimeter fence. Six people were injured. One person died of their injuries several days later."


# Analyse der Daten

In [13]:
%sql

select count(*) AS CrashesInSwitzerland 
from crashestable 
where lower(location) like '%switzerland%'

CrashesInSwitzerland
23


In [14]:
%sql

select count(*) as SwissRelatedCrashes
from crashestable 
where lower(operator) like 'swiss%' or
      lower(operator) like 'ju air'

SwissRelatedCrashes
13


In [15]:
%sql
drop table crashesred

In [16]:
%sql

create table crashesred
select date,
       time,
       location,
       case operator
         when 'Lufthansa' then 'Deutsche Lufthansa'
         when 'Deutche Lufthansa' then 'Deutsche Lufthansa'
         when 'Lufthansa Cityline' then 'Deutsche Lufthansa'
         when 'Lufthansa Cargo Airlines' then 'Deutsche Lufthansa'
         when 'Eurowings' then 'Deutsche Lufthansa'
         when 'Germanwings' then 'Deutsche Lufthansa'
         else operator
       end as operator,
       flight_no,
       route,
       ac_type,
       registration,
       cn_ln,
       aboard,
       fatalities,
       ground,
       summary
from crashestable
where operator not like '%Military%' and 
      operator not like '%USMC%' and
      operator not like '%Marines%' and
      operator not like '%Navy%' and
      operator not like '%Private%'

In [17]:
%sql 

select operator,
       count(*) as anz
from crashesred
group by operator 
order by anz desc
limit 10

operator,anz
Aeroflot,260
Deutsche Lufthansa,75
Air France,72
United Air Lines,44
China National Aviation Corporation,43
Pan American World Airways,41
American Airlines,37
US Aerial Mail Service,35
KLM Royal Dutch Airlines,34
Indian Airlines,34


In [18]:
%sql
select floor(year(date)/10)*10 as decade,
       operator,
       count(*) as anz
from crashesred
where operator in ('Deutsche Lufthansa','Aeroflot','Air France','United Airlines','China National Aviation Corporation')
group by operator, floor(year(date)/10)*10
order by anz desc

decade,operator,anz
1970,Aeroflot,92
1960,Aeroflot,59
1980,Aeroflot,45
1930,Deutsche Lufthansa,34
1940,China National Aviation Corporation,33
1950,Aeroflot,30
1930,Air France,22
1940,Deutsche Lufthansa,19
1940,Air France,18
1990,Aeroflot,18


In [19]:
%sql
drop table crashes_causes

In [20]:
%sql
create table crashes_causes
select *, 'Weather' as reason
from crashesred
where lower(summary) like '%severe turbulence%'  or
      lower(summary) like '%windshear%' or
      lower(summary) like '%mountain wave%' or
      lower(summary) like '%poor visibility%' or
      lower(summary) like '%heavy rain%' or 
      lower(summary) like '%severe winds%' or
      lower(summary) like '%icing%' or
      lower(summary) like '%thunderstorms%' or
      lower(summary) like '%lightning strike%'
union
select *, 'Sabotage' as reason
from crashesred
where lower(summary) like '%bomb%' or
      lower(summary) like '%hijack%' or
      lower(summary) like '%shot down%'
union
select *, 'Mechanical' as reason
from crashesred
where lower(summary) like '%engine failure%' or
      lower(summary) like '%equipment failure%' or
      lower(summary) like '%design flaw%' or
      lower(summary) like '%maintenance error%'      
union
select *, 'Pilot Error' as reason
from crashesred
where lower(summary) like '%improper procedure%' or
      lower(summary) like '%flying vfr into ifr conditions%' or
      lower(summary) like '%controlled flight into terrain%' or
      lower(summary) like '%descending below minima%' or
      lower(summary) like '%spatial disorientation%' or
      lower(summary) like '%premature descent%' or
      lower(summary) like '%excessive landing speed%' or
      lower(summary) like '%missed runway%' or
      lower(summary) like '%fuel starvation%' or
      lower(summary) like '%navigation error%' or
      lower(summary) like '%wrong runway takeoff/landing%' or
      lower(summary) like '%midair collision caused by both pilots%'
union
select *, 'Other' as reason
from crashesred
where lower(summary) like '%atc error%' or
      lower(summary) like '%ground crew error%' or
      lower(summary) like '%overloaded%' or
      lower(summary) like '%improperly loaded cargo%' or
      lower(summary) like '%bird strike%' or
      lower(summary) like '%fuel contamination%' or
      lower(summary) like '%pilot incapacitation%' or
      lower(summary) like '%obstruction on runway%' or
      lower(summary) like '%midair collision caused by other plane%' or
      lower(summary) like '%fire/smoke in flight%' or
      lower(summary) like '%fire in flight%' or
      lower(summary) like '%smoke in flight%'

In [21]:
%sql

select floor(year(date)/10)*10 as decade,
       reason,
       count(reason)
from crashes_causes
group by floor(year(date)/10)*10, reason
order by floor(year(date)/10)*10


decade,reason,count(reason)
1910,Sabotage,1
1920,Mechanical,8
1920,Weather,7
1920,Pilot Error,3
1930,Sabotage,4
1930,Weather,31
1930,Mechanical,10
1940,Pilot Error,7
1940,Mechanical,17
1940,Sabotage,29


Um heraus zu finden wie präzise dieser einfache Kategorisierungsansatz ist, wurden jeweils 50 Records herausgesucht, deren Kategorisierung wurden manuell überprüft, dabei hat sich folgendes herausgestellt:

- Keine Records ohne Kategorisierung
- Datensample A ergibt eine Präzision von 96% bei der Kategorisierung
- Datensample B ergibt eine Präzision von 98% bei der Kategorisierung

Somit zeigt sich das schon ein einfacher Ansatz zur Kategorisierung der Absturzursachen zu einer sehr hohen Präzision führt. Daher ist ein ML basierender Ansatz in diesem Fall gar nicht nötig.

In [23]:
%sql

select count(*) 
from crashes_causes 
where reason is null

/*Yippie keine Nuller*/

count(1)
0


In [24]:
%sql

select summary, reason, operator
from crashes_causes
order by year(date) desc
limit 50

summary,reason,operator
"The charter flight was just seconds into it's flight when it was observed flying low, before it nose diving into a building at a shopping center and bursting into flames. The pilot made two mayday calls before the aircraft crashed. It is reported the plane suffered engine failure.",Mechanical,Australian Corporate Jet Centres PTY. LTD
Shortly after leaving Mogadishu Airport a loud bang was heard in the cabin. There was a loss of pressurization and the plane made an emergency landing back at the airport. A large hole was observed in the side of the fuselage. A bomb containing TNT was detonated in the cabin. The bomber was sucked out of the plane and killed.,Sabotage,Daallo Airlines
"While on approach, the plane stalled and crased in a wooded area 1 km short of the runway. The accident took place in stormy weather.Three survived, but later died of their injuries. Possible engine failure.",Mechanical,Sunbird Aviation
"The army transport crashed while attempting to land in fog after hitting electric lines. The Al Nursa front, claimed that it shot down the aircraft.",Sabotage,Syrian Air Force
The passenger plane crashed into Tangok mountain located in a remote area of Papua's eastern most provence. There was no distress call from the aircraft. High winds and heavy rain was reported in the accident area.,Weather,Trigana Air Service
The aircraft crashed into the ocean shortly after taking off from Kalaupapa Airport. Preliminary indications are that there was catastrophic engine failure.,Mechanical,Makani Kai Air
"While landing in heavy rain, the aircraft veered off the runway, caught fire after hitting some brush and came to rest inverted.",Weather,Aerocon
"The cargo plane crashed about 20 miles northeast of Dillingham while on approach to Dillingham Airport. Wreckage was discovered on Saturday with the pilot and copilot fatally injured. The aircraft was badly fragmented in three sections. The flight crew's failure to maintain terrain clearance, which resulted in controlled flight into terrain in instrument meteorological conditions. Contributing to the accident were the flight crew's failure to correctly read back and interpret clearance altitudes issued by the air traffic controller, their failure to adhere to minimum altitudes depicted on the published instrument approach chart, and their failure to adhere to company checklists. Also contributing to the accident were the air traffic controller's issuance of an ambiguous clearance to the flight crew, which resulted in the airplane's premature descent, his failure to address the pilot's incorrect read back of the assigned clearance altitudes, and his failure to monitor the flight and address the altitude violations and issue terrain-based safety alerts.",Pilot Error,Ace Air Cargo
The airliner crashed into Bwabwata National Park while en route to Luanda. The plane was found completely burned out with no survivors. The plane went missing in bad weather and poor visibility.,Weather,Mozambique Airline
The aircraft crashed short of the runway while attempting to land in heavy rain at Goma Airport crashing into the back yards of several houses in the Lake Kivu area.,Weather,Compagnie Africaine d' Aviation (CAA)


In [25]:
%sql

select *
from crashes_causes
order by year(date) asc
limit 50

date,time,location,operator,flight_no,route,ac_type,registration,cn_ln,aboard,fatalities,ground,summary,reason
1917-06-17,?,"Near Yarmouth, England",?,?,?,Zepplin LZ-95 (air ship),L-48,?,16 (passengers:? crew:?),14 (passengers:? crew:?),0,Shot down by aircraft.,Sabotage
1920-09-27,?,"Hillersburg, Pennsylvania",US Aerial Mail Service,?,?,De Havilland DH-4,316,?,1 (passengers:0 crew:1),1 (passengers:0 crew:1),0,The plane struck power lines and crashed in flames after flying at a low altitude in poor visibility.,Weather
1920-06-06,?,"Cleveland, Ohio",US Aerial Mail Service,?,?,De Havilland DH-4,204,?,1 (passengers:0 crew:1),1 (passengers:0 crew:1),0,The aircraft suffered engine failure and crashed shortly after taking off.,Mechanical
1920-08-16,?,"College Park, Maryland",US Aerial Mail Service,?,?,De Havilland DH-4,317,?,1 (passengers:0 crew:1),1 (passengers:0 crew:1),0,"While approaching in poor visibility to land, the plane hit a radio mast and crashed in flames.",Weather
1920-03-30,?,"Newark, New Jersey",US Aerial Mail Service,?,Washington - Newark,De Havilland DH-4,72,?,1 (passengers:0 crew:1),1 (passengers:0 crew:1),0,George Sherlock was killed when his mail plane crashed into a chimney while attempting to land at Heller Field in poor visibility.,Weather
1920-06-20,?,"Cartagena, Columbia",Compañia Colombiana de Navegación Aérea,?,?,Farman F-40,?,?,1 (passengers:1 crew:1),1 (passengers:1 crew:1),0,Crashed while taking off after engine failure. Engine failure due to fuel exhaustion.,Mechanical
1922-04-07,1315,"Grandvilliers, France",Daimler Airways / Grands Express Aeriens,?,Croydon - Le Bourget,de Havilland DH-18 / Farman F-60 Goliath,G-EAOW/FGEAD,4,7 (passengers:5 crew:2),7 (passengers:5 crew:2),0,"A midair collision occurred at 400 ft. in poor visibility approximately 18 miles north of Beauvais, France. The pilots of both planes were using the road as a route marker in bad weather and poor visibility. This was the first commercial midair collision in aviation history. Two killed on the de Havilland, five on the Faman F-60.",Weather
1923-09-14,c 18:00,"Near Ivinghoe, Buckinghamshire, England",Daimler Airways,?,Croydon - Manchester,de Havilland DH-34,G-EBBS,29,5 (passengers:3 crew:2),5 (passengers:3 crew:2),0,Crashed while attempting a forced landing after encountering heavy rain.,Weather
1924-12-24,c 12:00,"Purley, England",Imperial Airways,?,London - Paris,de Havilland DH-34B,G-EBBX,36,8 (passengers:7 crew:1),8 (passengers:7 crew:1),0,"Shortly after taking off, witnesses observed a puff of white smoke after which the plane nose-dived into the ground. Fuel starvation. Captain Stewart and seven passengers were killed in an attempted forced landing.",Pilot Error
1925-04-25,?,"Alicante, Spain",Grands Express Aeriens,?,Alicante - Barcelona - Toulouse,Breguet 14,F-AFBB,202,3 (passengers:1 crew:2),3 (passengers:1 crew:2),0,"Shortly after takeoff, while flying at a low altitude, in poor visibility, the, aircraft hit a lightning rod, stalled and crashed.",Weather


#Zusätzlich den Flugzeugtype den grössten Flugzeugherstellern zuweisen

In [27]:
%sql

drop table crashes_manufacturer

In [28]:
%sql 

create table crashes_manufacturer
select *, 'Airbus' as Manufacturer
from crashes_causes
where lower(ac_type) like '%airbus%'
union
select *, 'ATR' as Manufacturer
from crashes_causes
where lower(ac_type) like '%atr%'
union
select *, 'Antonov' as Manufacturer
from crashes_causes
where lower(ac_type) like '%antonov%'
union
select *, 'Beechcraft' as Manufacturer
from crashes_causes
where lower(ac_type) like '%beech%'
union
select *, 'Boeing' as Manufacturer
from crashes_causes
where lower(ac_type) like '%boeing%'
union
select *, 'British Aerospace' as Manufacturer
from crashes_causes
where lower(ac_type) like '%british aerospace%' or
      lower(ac_type) like '%bae%'
union
select *, 'Cessna' as Manufacturer
from crashes_causes
where lower(ac_type) like '%cessna%'
union
select *, 'Embraer' as Manufacturer
from crashes_causes
where lower(ac_type) like '%embraer%'
union
select *, 'Fokker' as Manufacturer
from crashes_causes
where lower(ac_type) like '%fokker%'
union
select *, 'Ilyushin' as Manufacturer
from crashes_causes
where lower(ac_type) like '%ilyushin%'
union
select *, 'Junkers' as Manufacturer
from crashes_causes
where lower(ac_type) like '%junkers%'
union
select *, 'Lookheed Martin' as Manufacturer
from crashes_causes
where lower(ac_type) like '%lookheed%' or
      lower(ac_type) like '%martin%'
union
select *, 'McDonnell Douglas' as Manufacturer
from crashes_causes
where lower(ac_type) like '%douglas%' or
      lower(ac_type) like '%mcdonnell%' or
      lower(ac_type) like '%md%'
union
select *, 'Piper' as Manufacturer
from crashes_causes
where lower(ac_type) like '%piper%' 

In [29]:
%sql

select manufacturer,
       floor(year(date)/10)*10 as Decade,
       count(*) as Crashes
from crashes_manufacturer
group by manufacturer, floor(year(date)/10)*10
order by floor(year(date)/10)*10 asc

manufacturer,Decade,Crashes
Boeing,1920,1
Junkers,1920,1
McDonnell Douglas,1920,1
Fokker,1920,4
Lookheed Martin,1930,1
Junkers,1930,7
McDonnell Douglas,1930,5
Fokker,1930,2
Boeing,1930,2
Ilyushin,1940,2


In [30]:
%sql

select ac_type, count(ac_type)
from crashes_causes
group by ac_type
order by count(ac_type) desc

ac_type,count(ac_type)
Douglas DC-3,57
de Havilland Canada DHC-6 Twin Otter 300,20
Douglas DC-4,10
Antonov AN-26,9
Douglas DC-6B,7
Douglas C-47A,7
Ilyushin IL-12,6
Yakovlev YAK-40,6
Douglas DC-6,6
Junkers JU-52/3m,6
